# Scrape a Website Table into a Pandas DataFrame


__Goal:__ Scrape the New England Small College Athletic Conference (NESCAC) volleyball schedule from the NESCAC website into a `pandas` DataFrame. Save a cleaned conference schedule DataFrame and team conference schedule DataFrame to a comma-separated value (CSV) file.


We can scrape schedules from the NESCAC website in one of two ways:

* Directly through the website using a URL 
* Indirectly through a saved version of the website's HTML

The direct method involves fewer steps but is vulnerable because a change in the website's format could break our code. This means that even though our code works today, there is no guarantee our code will work tomorrow. In contrast, the indirect method adds the step of saving the website's HTML to sidestep the potential for our code to break in the future. 

In our situation, we are dealing with a small number of HTML files, so we prioritize code stability. We focus on scraping data from a saved version of the website's HTML.


## Scrape Schedule DataFrame from HTML File

We need to start with saving the HTML file. To do so, we open Google Chrome and navigate to the [2017 NESCAC Volleyball Schedule](http://www.nescac.com/sports/wvball/2017-18/schedule) website, right-click on the page, and click `Save As`.

<img src="images/1.png">

Not bad. Now, let's take a more in-depth look at the format of this schedule. Right-click somewhere on the table and click `Inspect`.

<img src="images/2.png">

This opens Chrome's DevTools, where the `Elements` tab displays the website's HTML. The HTML is organized in a tree structure, and we want to find the highest level of the tree that includes all elements of our schedule table. Hovering over the `<table>` tag, it appears we found our winner. A little more inspection reveals the `<tr>` tag identifies a table row and the `<td>` tag identifies the column values within each row. This information comes in handy shortly.

<img src="images/3.png">

Now to our Jupyter notebook. To begin, we import necessary libraries.

In [1]:
# Import necessary libraries
import pandas as pd
from bs4 import BeautifulSoup

We load in our HTML file and use `BeautifulSoup` to find the `<table>` and `<tr>` tags, creating a table variable and rows variable to store this information along the way. Note that we skip the first row containing column headers in the table (`[1:]`) because we set the column header values manually.

In [2]:
# Set schedule year to scrape
year = 2017

# Determine filename of saved websited HTML
html_file = 'hard_data/' + str(year) + '_nescac_schedule.html'

# Parse the HTML
with open(html_file, 'r') as fin:
    soup = BeautifulSoup(fin.read(), 'lxml')
    
# Extract the schedule table
table = soup.find_all('table')[0]

# Extract the table rows - exclude header rows
rows = table.find_all('tr')[1:]

We create a dictionary to hold a list of column values. We eventually use this dictionary to create our `pandas` DataFrame. 

In [3]:
# Initialize data dictionary to hold column values
data = {
    'Date': [],
    'Month': [],
    'Team 1': [],
    'Team 2': [],
    'Location': []
}

An alternative and slightly more intuitive method would initialize a DataFrame and create and append a row to that DataFrame for each row in the HTML table. The problem? `pandas` returns a copy of the original DataFrame plus your new row every time you use the append function. This gets slow quickly. With our small, fifty-five row schedule, creating the DataFrame with a dictionary takes approximately 378 ms. Using the append function to add a DataFrame row in each pass takes 471 ms. Using a dictionary is the way to go.

The NESCAC schedule table displays the month as a single value row, almost like a header for the matches below. The date is similar where the date is written one time, and matches without a date listed in the same row share the date above.

<img src="images/4.png">

We need to account for this in our `pandas` DataFrame because we want each match to have a value for a month and a date. To start, we initialize variables for the current date and the current month. We update these values shortly.

In [4]:
# Initialize current date and current month
curr_date = None
curr_month = None

We are ready to iterate over the rows in the table and store values in our column value dictionary. Let's break down this code piece by piece.

In [5]:
# Iterate over table rows
for row in rows:

    # Extract column values
    cols = row.find_all('td')

    # Check if row contains a month value
    if len(cols) == 1:

        # Set month value as current month
        curr_month = cols[0].get_text()

    # Check if row contains sufficient data
    elif len(cols) > 1:

        # Find value for date column
        date = cols[0].get_text().strip()

        # Check if date column contains date
        if date:

            # Set date value as current date
            curr_date = date

        # Find values for remaining relevant columns
        away_team = cols[1].get_text().strip()
        home_team = cols[2].get_text().strip()
        location = cols[3].get_text().strip()

        # Check if a conference match (*) not in post-season (%)
        if '*' in away_team and '%' not in away_team:

            # Add values to data dictionary
            data['Date'].append(curr_date)
            data['Month'].append(curr_month)
            data['Team 1'].append(home_team)
            data['Team 2'].append(away_team)
            data['Location'].append(location)

Looking at a single row, we use `BeautifulSoup` to find the `<td>` tags. As a reminder, the `<td>` tag identifies the column values within each row. We store this information in a `cols` variable.

We are on the lookout for a current month value. We know rows containing a month have only one column. Another way to say this, the length of cols == 1 for a current month row. So when we look at a table row, we need to check if it contains a month value. If it does, we update the value of the current month and move to the next row. Otherwise, we don't do anything and move to the next check. This updates the blank month values with the correct month in our DataFrame.

We now check that the row has more than one column. This indicates that we found a row containing match information. Since we are on the lookout for a current date value, we start our review of the match information by checking for a value in the date column (the first column in the table). If the date column contains data, we found a date value to use as our current date, and we update our current date variable. If not, we use a previously set current date value. This will fill in the blank date values with the correct date in our DataFrame.

We find values for the remaining columns: `away_team`, `home_team`, and `location`, and we use the `strip` function to remove leading and trailing whitespaces from these values. Extraneous characters (including whitespaces) clutter our DataFrame, so proactively removing them is a good idea.

The final check in our block of code ensures we only add data for conference matches (excluding postseason games) to our DataFrame. A quick scan of the NESCAC website reveals that an asterisk (`*`) in the away column identifies a conference match, and a percent sign (`%`) in the away column identifies a postseason match. Accordingly, in our code, we check if a row's `away` column contains an `*` and does not contain a `%`. If this is true, we found a non-postseason conference match and we store column values in our dictionary. If this is false, we move on to the next row.

Once we have iterated over all rows in the table, we can create and preview the resulting DataFrame. 

In [6]:
# Create uncleaned conference schedule DataFrame
schedule_df = pd.DataFrame(data)

# Preview DataFrame
schedule_df.head(10)

,Date,Month,Team 1,Team 2,Location
0,Fri. 15,September,Bowdoin\n3,Colby\n*\n0,"Brunswick, Maine"
1,Fri. 15,September,Tufts\n3,Bates\n*\n0,"Medford, Mass."
2,Fri. 15,September,Amherst\n2,Middlebury\n*\n3,"Amherst, Mass."
3,Fri. 15,September,Williams\n0,Wesleyan\n*\n3,"Williamstown, Mass."
4,Fri. 15,September,Hamilton\n3,Connecticut College\n*\n1,"Clinton, N.Y."
5,Sat. 16,September,Bowdoin\n3,Bates\n*\n1,"Brunswick, Maine"
6,Sat. 16,September,Tufts\n3,Colby\n*\n0,"Medford, Mass."
7,Sat. 16,September,Hamilton\n0,Wesleyan\n*\n3,"Clinton, N.Y."
8,Sat. 16,September,Trinity\n1,Middlebury\n*\n3,"Hartford, Conn."
9,Sat. 16,September,Williams\n3,Connecticut College\n*\n0,"Williamstown, Mass."


Not bad for a first pass, but there is still cleaning to do.

## Clean Team Columns

We'll clean the `Team 1` and `Team 2` columns first. We need to remove the extraneous newline characters (`\n`) and numbers. Let's use the `split` function. Looking at the first `Team 2` column value:

In [7]:
schedule_df['Team 2'][0]

'Colby\n*\n0'

We can split on the newline character:

In [8]:
schedule_df['Team 2'][0].split('\n')

['Colby', '*', '0']

And take the first value in the list:

In [9]:
schedule_df['Team 2'][0].split('\n')[0]

'Colby'

We want to apply this to the `Team 1` and `Team 2` columns, which we can do using the map function with a lambda function.

In [10]:
# Clean team 1 and team 2 columns
for column in ['Team 1', 'Team 2']:
    schedule_df[column] = schedule_df[column].map(
        lambda x: x.split('\n')[0])
    
# Preview DataFrame
schedule_df.head()

,Date,Month,Team 1,Team 2,Location
0,Fri. 15,September,Bowdoin,Colby,"Brunswick, Maine"
1,Fri. 15,September,Tufts,Bates,"Medford, Mass."
2,Fri. 15,September,Amherst,Middlebury,"Amherst, Mass."
3,Fri. 15,September,Williams,Wesleyan,"Williamstown, Mass."
4,Fri. 15,September,Hamilton,Connecticut College,"Clinton, N.Y."


Looks good. 

## Clean Location Column

Now let's work on the `Location` column. We'll switch our example schedule DataFrame to 2016 and preview the first 10 rows.

In [11]:
from src.scrape_schedule import *


schedule_df = scrape_uncleaned_schedule('2016')

# Preview DataFrame
schedule_df.head(10)

,Date,Month,Team 1,Team 2,Location
0,Fri. 16,September,Bowdoin,Amherst,"Brunswick, Maine"
1,Fri. 16,September,Connecticut College,Colby,"Green Dot Match New London, Conn."
2,Fri. 16,September,Wesleyan,Hamilton,"Middletown, Conn."
3,Fri. 16,September,Trinity,Middlebury,"Hartford, Conn."
4,Fri. 16,September,Tufts,Williams,"Medford, Mass."
5,Sat. 17,September,Trinity,Hamilton,"(25-19, 25-12, 24-22) Hartford, Conn."
6,Sat. 17,September,Tufts,Amherst,"Medford, Mass."
7,Sat. 17,September,Connecticut College,Bates,"New London, Conn."
8,Sat. 17,September,Bowdoin,Williams,"Brunswick, Maine"
9,Sat. 17,September,Wesleyan,Middlebury,"Middletown, Conn."


A few location values don't look like the others. We need to correct this to ensure all location values have a standard format.

We can correct the location at row index `1` with a one-off fix since it is the only row where this `Green Dot Match ...` location occurs. We create a dictionary with the erroneous location as the key and the corrected location as the value, and we use the `replace` function to apply the changes. (As a side note, the `locations_to_replace` dictionary also includes a `Waterville, Maine.` correction needed in the 2015 schedule at row index `46`.)

In [12]:
# Create dictionary of one-off location values to change
locations_to_replace = {
    "Green Dot Match  New London, Conn.": "New London, Conn.",
    "Waterville, Maine.": "Waterville, Maine"
}

# Replace one-off location values in DataFrame
schedule_df['Location'] = schedule_df['Location'].replace(locations_to_replace)

# Preview DataFrame
schedule_df.head(10)

,Date,Month,Team 1,Team 2,Location
0,Fri. 16,September,Bowdoin,Amherst,"Brunswick, Maine"
1,Fri. 16,September,Connecticut College,Colby,"New London, Conn."
2,Fri. 16,September,Wesleyan,Hamilton,"Middletown, Conn."
3,Fri. 16,September,Trinity,Middlebury,"Hartford, Conn."
4,Fri. 16,September,Tufts,Williams,"Medford, Mass."
5,Sat. 17,September,Trinity,Hamilton,"(25-19, 25-12, 24-22) Hartford, Conn."
6,Sat. 17,September,Tufts,Amherst,"Medford, Mass."
7,Sat. 17,September,Connecticut College,Bates,"New London, Conn."
8,Sat. 17,September,Bowdoin,Williams,"Brunswick, Maine"
9,Sat. 17,September,Wesleyan,Middlebury,"Middletown, Conn."


All set. 

To correct the location at row index `5`, we need to keep only the characters after the match scores. Looking at this location more closely:

In [13]:
schedule_df['Location'][5]

'(25-19, 25-12, 24-22) Hartford, Conn.'

We can split on the `)` character:

In [14]:
schedule_df['Location'][5].split(')')

['(25-19, 25-12, 24-22', ' Hartford, Conn.']

And take the last value in the list with leading and trailing white spaces stripped:

In [15]:
schedule_df['Location'][5].split(')')[-1].strip()

'Hartford, Conn.'

We again use the `map` function with a `lambda` function to apply this to the `Location` column.

In [16]:
# Remove match scores with parenthesis
schedule_df['Location'] = schedule_df['Location'].map(
    lambda x: x.split(')')[-1].strip())

# Preview DataFrame
schedule_df.head(10)

,Date,Month,Team 1,Team 2,Location
0,Fri. 16,September,Bowdoin,Amherst,"Brunswick, Maine"
1,Fri. 16,September,Connecticut College,Colby,"New London, Conn."
2,Fri. 16,September,Wesleyan,Hamilton,"Middletown, Conn."
3,Fri. 16,September,Trinity,Middlebury,"Hartford, Conn."
4,Fri. 16,September,Tufts,Williams,"Medford, Mass."
5,Sat. 17,September,Trinity,Hamilton,"Hartford, Conn."
6,Sat. 17,September,Tufts,Amherst,"Medford, Mass."
7,Sat. 17,September,Connecticut College,Bates,"New London, Conn."
8,Sat. 17,September,Bowdoin,Williams,"Brunswick, Maine"
9,Sat. 17,September,Wesleyan,Middlebury,"Middletown, Conn."


Perfect.

Let's look at the schedule DataFrame for 2015 for another type of location value that needs cleaning. 

In [43]:
from src.scrape_schedule import *


schedule_df = scrape_uncleaned_schedule('2015')

# Preview DataFrame
schedule_df[40:50]

,Date,Month,Team 1,Team 2,Location
40,Sat. 17,October,Williams,Bowdoin,"Williamstown, Mass."
41,Sat. 17,October,Colby,Hamilton,"Waterville, Maine"
42,Sat. 17,October,Connecticut College,Wesleyan,"New London, Conn."
43,Sat. 17,October,Tufts,Trinity,"Medford, Mass."
44,Sat. 17,October,Bates,Middlebury,"Lewiston, Maine"
45,Fri. 30,October,Trinity,Amherst,"25-16, 25-13, 25-19 Hartford, Conn."
46,Fri. 30,October,Colby,Bowdoin,"Waterville, Maine."
47,Fri. 30,October,Middlebury,Connecticut College,"Middlebury, Vt."
48,Fri. 30,October,Hamilton,Tufts,"Clinton, N.Y."
49,Fri. 30,October,Wesleyan,Williams,"Middletown, Conn."


The `Location` value at row index `45` includes match scores without parenthesis. Without parenthesis, our prior fix to remove match scores will not work. There is no `)` to split on. 

We need another solution. Looking at our `Location` value:

In [44]:
schedule_df['Location'][45]

'25-16, 25-13, 25-19 Hartford, Conn.'

We first remove the commas preceded by a digit (we want the comma between the city and state to remain in place). To do this, we use a regular expression (regex) with look-behind that replaces all commas (`,`) preceded (`?<=`) by a digit (`\d`) with a blank character (`''`).

In [45]:
# Remove commas if preceded by a digit
schedule_df['Location'] = schedule_df['Location'].str.replace(
    '(?<=\d),', '')

# Example value
schedule_df['Location'][45]

'25-16 25-13 25-19 Hartford, Conn.'

Here's an excellent resource for more information about regex look-behind:

https://www.regular-expressions.info/lookaround.html

We use another regex statement to replace all (`+`) hyphens (`\-`) and digits (`\d`) with a blank character (`''`).

In [46]:
# Remove digits and hyphens
schedule_df['Location'] = schedule_df['Location'].str.replace(
    '[\-\d]+', '')

# Example value
schedule_df['Location'][45]

'   Hartford, Conn.'

We can now remove the leading and trailing white spaces to give a clean result.

In [47]:
# Remove leading and trailing whitespaces
schedule_df['Location'] = schedule_df['Location'].map(
    lambda x: x.strip())

# Example value
schedule_df['Location'][45]

'Hartford, Conn.'

Not bad. 

We ultimately want to use our scraped NESCAC schedule in correspondence with another NESCAC DataFrame. For this to work, we need the venue names from both DataFrames to align. 

We first create a dictionary of the city, state to venue name mappings, and use the `replace` function to implement the change.

In [48]:
# Create dictionary for location city, state to venue name mapping
locations_mapping = {
    'Brunswick, Maine': 'Bowdoin',
    'Medford, Mass.': 'Tufts',
    'Amherst, Mass.': 'Amherst',
    'Williamstown, Mass.': 'Williams',
    'Clinton, N.Y.': 'Hamilton',
    'Hartford, Conn.': 'Trinity',
    'Middletown, Conn.': 'Wesleyan',
    'Waterville, Maine': 'Colby',
    'New London, Conn.': 'Connecticut College',
    'Middlebury, Vt.': 'Middlebury',
    'Lewiston, Maine': 'Bates'
}

# Map location city, state to venue name in DataFrame
schedule_df = schedule_df.replace(locations_mapping)

# Preview DataFrame
schedule_df.head()

,Date,Month,Team 1,Team 2,Location
0,Fri. 18,September,Williams,Tufts,Williams
1,Fri. 18,September,Amherst,Connecticut College,Amherst
2,Fri. 18,September,Bates,Wesleyan,Bates
3,Fri. 18,September,Middlebury,Bowdoin,Middlebury
4,Fri. 18,September,Colby,Trinity,Colby


We now have `Team 1`, `Team 2`, and `Location` values using the same set of venue names. 


## Clean Team & Venue Names

Let's check out what the venue names look like in our other NESCAC DataFrame. We have the names stored in a [venues mapping](hard_data/venues_with_mapping.txt) file. Let's load the text file and make a list of "expanded" venue names.

In [49]:
# Load expanded venue names from file
with open("hard_data/venues_with_mapping.txt", 'r') as f:
    
    # Create list of expanded venue names
    expanded_names = [line.split(',')[0] for line in f] 

# Preview list of expanded venue names
expanded_names

['Amherst College',
 'Bates College',
 'Bowdoin College',
 'Colby College',
 'Connecticut College',
 'Hamilton College',
 'Middlebury College',
 'Trinity College',
 'Tufts University',
 'Wesleyan University',
 'Williams College']

It looks like we need to make some changes to our schedule venue names. We create a dictionary with old venue names as keys and new venue names as values. 

In [50]:
# Create list of abbreviated names from DataFrame
abbreviated_names = list(schedule_df['Team 1'].unique())

# Create abbreviated to expanded venue names mapping dictionary
abbreviated_to_expanded_mapping = dict(zip(
    sorted(abbreviated_names),
    sorted(expanded_names)))

# Preview dictionary
abbreviated_to_expanded_mapping

{'Amherst': 'Amherst College',
 'Bates': 'Bates College',
 'Bowdoin': 'Bowdoin College',
 'Colby': 'Colby College',
 'Connecticut College': 'Connecticut College',
 'Hamilton': 'Hamilton College',
 'Middlebury': 'Middlebury College',
 'Trinity': 'Trinity College',
 'Tufts': 'Tufts University',
 'Wesleyan': 'Wesleyan University',
 'Williams': 'Williams College'}

Notice the Connecticut College dictionary entry has the same key and value. We update the dictionary to exclude the Connecticut College key-value pair to avoid an error (replacement not allowed with overlapping keys and values).

In [51]:
# Update dictionary to avoid key: value matching error
abbreviated_to_expanded_mapping = {k: v for k, v in abbreviated_to_expanded_mapping.items() if k != v}

# Preview dictionary
abbreviated_to_expanded_mapping

{'Amherst': 'Amherst College',
 'Bates': 'Bates College',
 'Bowdoin': 'Bowdoin College',
 'Colby': 'Colby College',
 'Hamilton': 'Hamilton College',
 'Middlebury': 'Middlebury College',
 'Trinity': 'Trinity College',
 'Tufts': 'Tufts University',
 'Wesleyan': 'Wesleyan University',
 'Williams': 'Williams College'}

We can now replace the `Team 1`, `Team 2`, and `Location` column values in our schedule DataFrame with the correct venue names. 

In [52]:
# Replace abbreviated with expanded team and venue names in DataFrame
schedule_df = schedule_df.replace({
    'Team 1': abbreviated_to_expanded_mapping,
    'Team 2': abbreviated_to_expanded_mapping,
    'Location': abbreviated_to_expanded_mapping})

# Preview DataFrame
schedule_df.head(10)

,Date,Month,Team 1,Team 2,Location
0,Fri. 18,September,Williams College,Tufts University,Williams College
1,Fri. 18,September,Amherst College,Connecticut College,Amherst College
2,Fri. 18,September,Bates College,Wesleyan University,Bates College
3,Fri. 18,September,Middlebury College,Bowdoin College,Middlebury College
4,Fri. 18,September,Colby College,Trinity College,Colby College
5,Sat. 19,September,Amherst College,Tufts University,Amherst College
6,Sat. 19,September,Bates College,Trinity College,Bates College
7,Sat. 19,September,Hamilton College,Bowdoin College,Hamilton College
8,Sat. 19,September,Colby College,Wesleyan University,Colby College
9,Sat. 19,September,Williams College,Connecticut College,Williams College


Looks good.

## Clean Date-Related Columns

We are ready to clean our next columns - the `Month` and `Date` columns. Ultimately in our schedule DataFrame, we want a string abbreviated day of the week column and a date in `yyyy-mm-dd` format column. To create the string `Day (Str)` column, we use regex to extract all (`+`) non-digit characters (`\D`) from the `Date` column.

In [53]:
# Create day (str) column as all non-digit characters extracted
# from date column
schedule_df['Day (Str)'] = schedule_df['Date'].str.extract(
    '(\D+)', expand=True)

# Preview DataFrame
schedule_df.head()

,Date,Month,Team 1,Team 2,Location,Day (Str)
0,Fri. 18,September,Williams College,Tufts University,Williams College,Fri.
1,Fri. 18,September,Amherst College,Connecticut College,Amherst College,Fri.
2,Fri. 18,September,Bates College,Wesleyan University,Bates College,Fri.
3,Fri. 18,September,Middlebury College,Bowdoin College,Middlebury College,Fri.
4,Fri. 18,September,Colby College,Trinity College,Colby College,Fri.


We use the `pandas` built-in `to_datetime` function to create our formatted `Date` column. The function requires integer `Year`, `Month`, and `Day` columns. 

We create the `Year` column by setting all values equal to our year variable.

In [54]:
# Create year column
schedule_df['Year'] = year

# Preview DataFrame
schedule_df.head()

,Date,Month,Team 1,Team 2,Location,Day (Str),Year
0,Fri. 18,September,Williams College,Tufts University,Williams College,Fri.,2015
1,Fri. 18,September,Amherst College,Connecticut College,Amherst College,Fri.,2015
2,Fri. 18,September,Bates College,Wesleyan University,Bates College,Fri.,2015
3,Fri. 18,September,Middlebury College,Bowdoin College,Middlebury College,Fri.,2015
4,Fri. 18,September,Colby College,Trinity College,Colby College,Fri.,2015


We create our `Month` column by building a month mapping dictionary with the string month as the key and the integer month as the value. We then use the `map` function to apply the mapping to the `Month` column.

In [55]:
# Create dictionary for month mapping
month_mapping = {
    "September": 9,
    "October": 10,
    "November": 11
}

# Map string month to integer month
schedule_df['Month'] = schedule_df['Month'].map(month_mapping)

# Preview DataFrame
schedule_df.head()

,Date,Month,Team 1,Team 2,Location,Day (Str),Year
0,Fri. 18,9,Williams College,Tufts University,Williams College,Fri.,2015
1,Fri. 18,9,Amherst College,Connecticut College,Amherst College,Fri.,2015
2,Fri. 18,9,Bates College,Wesleyan University,Bates College,Fri.,2015
3,Fri. 18,9,Middlebury College,Bowdoin College,Middlebury College,Fri.,2015
4,Fri. 18,9,Colby College,Trinity College,Colby College,Fri.,2015


We create our integer `Day` column by using regex to extract all (`+`) digits (`\d`) from the `Date` column. 

In [56]:
# Create day column as all digit characters extracted from date
# column
schedule_df['Day'] = schedule_df['Date'].str.extract(
    '(\d+)', expand=True)

# Preview DataFrame
schedule_df.head()

,Date,Month,Team 1,Team 2,Location,Day (Str),Year,Day
0,Fri. 18,9,Williams College,Tufts University,Williams College,Fri.,2015,18
1,Fri. 18,9,Amherst College,Connecticut College,Amherst College,Fri.,2015,18
2,Fri. 18,9,Bates College,Wesleyan University,Bates College,Fri.,2015,18
3,Fri. 18,9,Middlebury College,Bowdoin College,Middlebury College,Fri.,2015,18
4,Fri. 18,9,Colby College,Trinity College,Colby College,Fri.,2015,18


And finally, we use the `pandas` `to_datetime` function and overwrite our `Date` column as the `yyyy-mm-dd` format of our date.

In [57]:
# Create date column as date in yyyy-mm-dd format
schedule_df['Date'] = pd.to_datetime(
    schedule_df[['Year', 'Month', 'Day']])

# Preview DataFrame
schedule_df.head()

,Date,Month,Team 1,Team 2,Location,Day (Str),Year,Day
0,2015-09-18,9,Williams College,Tufts University,Williams College,Fri.,2015,18
1,2015-09-18,9,Amherst College,Connecticut College,Amherst College,Fri.,2015,18
2,2015-09-18,9,Bates College,Wesleyan University,Bates College,Fri.,2015,18
3,2015-09-18,9,Middlebury College,Bowdoin College,Middlebury College,Fri.,2015,18
4,2015-09-18,9,Colby College,Trinity College,Colby College,Fri.,2015,18


We remove unnecessary date-related columns to finalize the DataFrame.

In [58]:
# Remove unneccesary date-related columns from DataFrame
schedule_df = schedule_df.drop(['Year', 'Month', 'Day'], axis=1)

# Preview DataFrame
schedule_df.head()

,Date,Team 1,Team 2,Location,Day (Str)
0,2015-09-18,Williams College,Tufts University,Williams College,Fri.
1,2015-09-18,Amherst College,Connecticut College,Amherst College,Fri.
2,2015-09-18,Bates College,Wesleyan University,Bates College,Fri.
3,2015-09-18,Middlebury College,Bowdoin College,Middlebury College,Fri.
4,2015-09-18,Colby College,Trinity College,Colby College,Fri.


It looks like our schedule DataFrame is all set.


## Create Team Schedule DataFrame

Now we can make a team schedule DataFrame where the column `Team 1` holds the team and the column `Team 2` holds the opponent for each team's matches. 

We initialize a `team_schedule` list to hold match information for each row, and we iterate over rows in our schedule DataFrame. We create a dictionary containing each row's match information, and we store each dictionary in the `team_schedule` list. We then switch the `Team 1` and `Team 2` values in our dictionary and save to the `team_schedule` list again. This ensures our team schedule contains all of each team's matches, whether the team was a `Team 1` or a `Team 2`.

In [59]:
# Initialize list to hold match information
team_schedule = []

for idx, row in schedule_df.iterrows():

    # Create dictionary with match information
    match = {column: row[column] for column in list(schedule_df.columns)}

    # Store in list
    team_schedule.append(match)

    # Copy dictionary with match information
    switched = match.copy()

    # Switch team 1 and team 2 match information
    switched['Team 1'] = row['Team 2']
    switched['Team 2'] = row['Team 1']

    # Store in list
    team_schedule.append(switched)

Once we iterate over all rows, we create a team schedule DataFrame and sort the values by `Team 1`, `Date`, and `Location`.

In [60]:
# Create team conference schedule DataFrame
team_schedule_df = pd.DataFrame(team_schedule)

# Sort values in NESCAC schedule
team_schedule_df = team_schedule_df.sort_values(['Team 1', 'Date', 'Location']).reset_index(drop=True)

# Preview DataFrame
team_schedule_df.head(15)

,Date,Team 1,Team 2,Location,Day (Str)
0,2015-09-18,Amherst College,Connecticut College,Amherst College,Fri.
1,2015-09-19,Amherst College,Tufts University,Amherst College,Sat.
2,2015-09-23,Amherst College,Williams College,Williams College,Wed.
3,2015-10-02,Amherst College,Colby College,Amherst College,Fri.
4,2015-10-03,Amherst College,Bates College,Amherst College,Sat.
5,2015-10-09,Amherst College,Middlebury College,Middlebury College,Fri.
6,2015-10-10,Amherst College,Hamilton College,Hamilton College,Sat.
7,2015-10-16,Amherst College,Bowdoin College,Amherst College,Fri.
8,2015-10-30,Amherst College,Trinity College,Trinity College,Fri.
9,2015-10-31,Amherst College,Wesleyan University,Wesleyan University,Sat.


Looks good. 


## Save Schedule DataFrames
As a final step, we save our DataFrames to CSV so we can access them elsewhere.

In [113]:
# Save schedule to CSV
schedule_df.to_csv('data/' + str(year) + '_nescac_schedule.csv')

# Save team schedule to CSV
team_schedule_df.to_csv('data/' + str(year) + '_nescac_team_schedule.csv')

Done!

## _Optional: Scrape Multiple Schedules in One Pass_

One final thought. Let's say we want to create multiple years of schedule and team schedule DataFrames in one pass. The backbone of the code is the same, but we create a list of years and HTML filenames.

In [114]:
# List of schedule years to scrape
years = list(range(2009, 2018))

# List of saved NESCAC website HTML files
html_filenames = ["hard data/" + str(year) + "_nescac_schedule.html" for year in years]

We iterate over the list of HTML filenames and create and save a schedule and team schedule for each year. (The code after the ellipsis is the same as the code discussed above).

In [ ]:
# Iterate over each schedule year to scrape
for i in range(len(years)):

    # Set schedule year to scrape
    year = years[i]
    
    # Determine filename of saved website HTML
    html_file = html_filenames[i]
    
    # Parse the HTML
    with open(html_file, 'r') as fin:
        soup = BeautifulSoup(fin.read(), 'lxml')

    # Extract the schedule table
    table = soup.find_all('table')[0]

    # Extract the table rows - exclude header rows
    rows = table.find_all('tr')[1:]

    ...